In [112]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\singh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [113]:
df = pd.read_csv('spam.csv',encoding='ISO-8859-1')
df.head()

v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [114]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis='columns',inplace=True)
df.columns = ['label', 'message']
df.head()


label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

In [115]:
ps = PorterStemmer()

In [116]:
corpus = []
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['message'][i])
    review = review.lower()
    review = nltk.word_tokenize(review)
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    
    

In [117]:
from sklearn.feature_extraction.text import CountVectorizer

In [118]:
cv = CountVectorizer(max_features = 5000)
X = cv.fit_transform(corpus).toarray()


In [119]:
y = pd.get_dummies(df['label'])
y = y.iloc[:,1].values

In [120]:
print(df['label'].head(),"\n y = ",y)

0     ham
1     ham
2    spam
3     ham
4     ham
Name: label, dtype: object 
 y =  [False False  True ... False False False]


In [121]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=0)

In [122]:
from sklearn.naive_bayes import MultinomialNB

In [123]:
spam_detect_model = MultinomialNB()
spam_detect_model.fit(X_train,y_train)




MultinomialNB()

In [124]:
y_pred = spam_detect_model.predict(X_test)

In [125]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test,y_pred)
cm

array([[938,  11],
       [  7, 159]], dtype=int64)

In [97]:
acc_score = accuracy_score(y_test,y_pred)
acc_score

0.9838565022421525